In [2]:
import glob
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

In [3]:
variable = "T2MIN"
nc_files_obs = glob.glob(f"data/OBS/*_{variable}*.nc")
nc_files_obs

['data/OBS\\OBS_monthly_T2MIN_1980-01-01-00_1980-12-31-18.nc',
 'data/OBS\\OBS_monthly_T2MIN_1981-01-01-00_1981-12-31-18.nc',
 'data/OBS\\OBS_monthly_T2MIN_1982-01-01-00_1982-12-31-18.nc',
 'data/OBS\\OBS_monthly_T2MIN_1983-01-01-00_1983-12-31-18.nc',
 'data/OBS\\OBS_monthly_T2MIN_1984-01-01-00_1984-12-31-18.nc',
 'data/OBS\\OBS_monthly_T2MIN_1985-01-01-00_1985-12-31-18.nc',
 'data/OBS\\OBS_monthly_T2MIN_1986-01-01-00_1986-12-31-18.nc',
 'data/OBS\\OBS_monthly_T2MIN_1987-01-01-00_1987-12-31-18.nc',
 'data/OBS\\OBS_monthly_T2MIN_1988-01-01-00_1988-12-31-18.nc',
 'data/OBS\\OBS_monthly_T2MIN_1989-01-01-00_1989-12-31-18.nc',
 'data/OBS\\OBS_monthly_T2MIN_1990-01-01-00_1990-12-31-18.nc',
 'data/OBS\\OBS_monthly_T2MIN_1991-01-01-00_1991-12-31-18.nc',
 'data/OBS\\OBS_monthly_T2MIN_1992-01-01-00_1992-12-31-18.nc',
 'data/OBS\\OBS_monthly_T2MIN_1993-01-01-00_1993-12-31-18.nc',
 'data/OBS\\OBS_monthly_T2MIN_1994-01-01-00_1994-12-31-18.nc',
 'data/OBS\\OBS_monthly_T2MIN_1995-01-01-00_1995-12-31-

In [4]:
lon_min, lon_max, lat_min, lat_max = -105.0, -89.0, 36.0, 45.0
# Open multiple NetCDF files as a single dataset
ds = xr.open_mfdataset(nc_files_obs, combine='nested', concat_dim="time")
ds

C:\Users\Meteo\AppData\Local\Temp\ipykernel_9204\2504917596.py:3: FutureWarning: In a future version of xarray the default value for data_vars will change from data_vars='all' to data_vars=None. This is likely to lead to different results when multiple datasets have matching variables with overlapping values. To opt in to new defaults and get rid of these warnings now use `set_options(use_new_combine_kwarg_defaults=True) or set data_vars explicitly.
  ds = xr.open_mfdataset(nc_files_obs, combine='nested', concat_dim="time")


<xarray.Dataset> Size: 233MB
Dimensions:  (time: 540, south_north: 138, west_east: 195, crs: 1)
Coordinates:
  * crs      (crs) uint16 2B 3
  * time     (time) datetime64[ns] 4kB 1980-01-31 1980-02-29 ... 2024-12-31
Dimensions without coordinates: south_north, west_east
Data variables:
    T2MIN    (time, south_north, west_east) float64 116MB dask.array<chunksize=(12, 138, 195), meta=np.ndarray>
    lat      (time, south_north, west_east) float32 58MB dask.array<chunksize=(12, 138, 195), meta=np.ndarray>
    lon      (time, south_north, west_east) float32 58MB dask.array<chunksize=(12, 138, 195), meta=np.ndarray>
Attributes:
    regrid_method:  bilinear

In [5]:
# Extract lon/lat variables
lon = ds["lon"]
lat = ds["lat"]

mask = (
    (lon >= lon_min) & (lon <= lon_max) &
    (lat >= lat_min) & (lat <= lat_max)
).compute()

# Apply mask
ds_subset = ds.where(mask, drop=True)
# Promote lat/lon to coordinates
ds_subset = ds_subset.set_coords(["lon", "lat"])
ds_subset

<xarray.Dataset> Size: 13MB
Dimensions:  (time: 540, south_north: 33, west_east: 47, crs: 1)
Coordinates:
    lat      (time, south_north, west_east) float32 3MB dask.array<chunksize=(12, 33, 47), meta=np.ndarray>
    lon      (time, south_north, west_east) float32 3MB dask.array<chunksize=(12, 33, 47), meta=np.ndarray>
  * crs      (crs) uint16 2B 3
  * time     (time) datetime64[ns] 4kB 1980-01-31 1980-02-29 ... 2024-12-31
Dimensions without coordinates: south_north, west_east
Data variables:
    T2MIN    (time, south_north, west_east) float64 7MB dask.array<chunksize=(12, 33, 47), meta=np.ndarray>
Attributes:
    regrid_method:  bilinear

In [6]:
# Save to a new NetCDF file
ds_subset.to_netcdf(f"data/OBS_subset/{variable}_subsetted_Midwest_US.nc")